In [ ]:
# USE_SAME_FEW_SHOTS=True

In [ ]:
num_shots = 5

In [ ]:
experiment_name = "2024.05.14-05_Phi-3-mini-128k-instruct_5-few-shot_SEMANTIC-SIMILARITY_WITH_DC"

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes_with_2023.12.11_input_data.pickle"

output_data_path = base_path + "/../Phi-3_mini/data/output/" + experiment_name
log_filename = base_path + "/../Phi-3_mini/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import pickle

In [ ]:
!pip install -q accelerate bitsandbytes rich transformers --progress-bar off

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig

In [ ]:
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(3)

# data_sel_4_few_shot_examples

In [ ]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"SDG-"+str(row['sdg'])+'\"\n\n'

#     few_shots += input_text
#     few_shots += output_text

# print(few_shots)

In [ ]:
with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [ ]:
data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

## Run the model

In [ ]:
data_sel = data

In [ ]:
text_id = data_sel.iloc[0].text_id
text = data_sel.iloc[0].text.replace('\n','').replace('{','(').replace('}',')')
sdg_true = data_sel.iloc[0].sdg

print("EXPECTED_SDG:", sdg_true, "\nTEXT:", text.replace('\n',''),'\n\n')

few_shots_list = top7_train_data_sim_to_each_test_data[text_id][:num_shots]
few_shots = ""
for train_data_id in few_shots_list:
  train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]
  input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
  output_text = "Output: \"SDG-"+str(train_row.sdg.values[0])+'\"\n\n'
  few_shots += input_text
  few_shots += output_text
print(few_shots)

template = f"""Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

"SDG-1": "End poverty in all its forms everywhere."
"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
"SDG-5": "Achieve gender equality and empower all women and girls."
"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
"SDG-10": "Reduce inequality within and among countries."
"SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
"SDG-12": "Ensure sustainable consumption and production patterns."
"SDG-13": "Take urgent action to combat climate change and its impacts."
"SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
"SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
"SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
"SDG-0": "Other."

Choose ONLY ONE label for each input text.

DO NOT include the input text in your answer.

The following are """+str(num_shots)+f""" examples of expected classification given an input text:

"""+few_shots+f"""

The input text is:
'''
{text}
'''
"""

messages = [
    {"role": "system", "content": "You are a text classifier."},
    {"role": "user", "content": template}
]

EXPECTED_SDG: 0 
TEXT: L train resumes service after derailing from track in Canarsie, Brooklyn  WABC-TVService restored after subway train derails in Brooklyn: officials  PIX11 New York NewsNo injuries in Brooklyn subway derailment  New York Daily NewsView Full Coverage on Google News. 


Input text: "NYC crime: 3 violent attacks on subways in span of 8 hours in Brooklyn, Queens; No arrests made  WABC-TVView Full Coverage on Google News."
Output: "SDG-0"

Input text: "Old trams and trolleybus lines are progressively disconnected because of the poor service and high operation and maintenance costs. In most cities, public transport relies on buses, and the light-rail transit (LRT) networks suffer from serious lack of rationalisation. Delegation of communal services to the private sector will work only if the national government helps city governments modernise communal infrastructure. However, city akimats need to be more pro-active in finding ways to refurbish urban infrastructure."
Ou

In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 "SDG-0"


In [ ]:
# STOP!

In [ ]:
data_sel = data #.iloc[785:]

In [ ]:
results = []

for index, row in data_sel.iterrows():

  cnt = index

  text_id = row['text_id']
  text = row["text"].replace('\n','').replace('{','(').replace('}',')')
  sdg_true = row["sdg"]

  file = open(log_filename, 'a+', encoding='utf-8')

  print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')
  file.write(f"EXPECTED_SDG: {row['sdg']} TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  # INIZIO AGGIUNTA PER SEMANTIC SIMILARITY
  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:num_shots]
  few_shots = ""
  for train_data_id in few_shots_list:
    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]
    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"SDG-"+str(train_row.sdg.values[0])+'\"\n\n'
    few_shots += input_text
    few_shots += output_text
  print(few_shots)
  # FINE AGGIUNTA PER SEMANTIC SIMILARITY

  template = f"""Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

"SDG-1": "End poverty in all its forms everywhere."
"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
"SDG-5": "Achieve gender equality and empower all women and girls."
"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
"SDG-10": "Reduce inequality within and among countries."
"SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
"SDG-12": "Ensure sustainable consumption and production patterns."
"SDG-13": "Take urgent action to combat climate change and its impacts."
"SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
"SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
"SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
"SDG-0": "Other."

Choose ONLY ONE label for each input text.

DO NOT include the input text in your answer.

The following are """+str(num_shots)+f""" examples of expected classification given an input text:

"""+few_shots+f"""

The input text is:
'''
{text}
'''
"""

  messages = [
    {"role": "system", "content": "You are a text classifier."},
    {"role": "user", "content": template}
  ]

  output = pipe(messages, **generation_args)

  result_extended = (cnt, text_id, sdg_true, output[0]['generated_text'])

  print(result_extended)
  file.write(str(result_extended)+'\n')

  with open(output_data_path+f'/sdg_classification_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  file.close()




##################################################
EXPECTED_SDG: 0 TEXT: L train resumes service after derailing from track in Canarsie, Brooklyn  WABC-TVService restored after subway train derails in Brooklyn: officials  PIX11 New York NewsNo injuries in Brooklyn subway derailment  New York Daily NewsView Full Coverage on Google News. 


Input text: "NYC crime: 3 violent attacks on subways in span of 8 hours in Brooklyn, Queens; No arrests made  WABC-TVView Full Coverage on Google News."
Output: "SDG-0"

Input text: "Old trams and trolleybus lines are progressively disconnected because of the poor service and high operation and maintenance costs. In most cities, public transport relies on buses, and the light-rail transit (LRT) networks suffer from serious lack of rationalisation. Delegation of communal services to the private sector will work only if the national government helps city governments modernise communal infrastructure. However, city akimats need to be more pro-active in

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Streaming output truncated to the last 5000 lines.

Input text: "In several natural resource-rich developing countries, infrastructure investments have pulled the economy in the direction of resource extraction, at the expense of other productive activity. For example, Mesquita Moreira et al. ( Like the balanced growth theorists, he recognized that externalities could disrupt any desired investment sequence."
Output: "SDG-9"

Input text: "Building on the success of the PPCDAm, the government launched a similar programme to control deforestation in the Cerrado biome. In addition to strict monitoring and enforcement, further reducing deforestation will increasingly require making sustainable agriculture and forest management more attractive than illegal logging and land grabbing, and ensuring that sustainable practices provide a viable income source for traditional communities (CEPAL et al., Concessions still cover less than 1% of the eligible forest area, mainly because of red tape, hig

In [ ]:
import pandas as pd

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

1020

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

,cnt,text_id,sdg_true,result
0,0,6668,0,"""SDG-0"""
1,1,6099,0,"""SDG-0"""
2,2,6471,0,"""SDG-16"""


In [ ]:
results_processed.iloc[0].result

' "SDG-0"'

In [ ]:
#results_processed.iloc[0].result.split("Answer with label only usign the format:---LABEL---")[1]
# results_processed['result_short'] = results_processed['result'].apply(lambda x: x.split("Answer with label only usign the format:---LABEL---")[1])

In [ ]:
# results_processed['result']

In [ ]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
# results_processed['label'] = results_processed['result_short'].str.extract(r'SDG (\d+)')

# results_processed['label'] = results_processed['result'].apply(
#     lambda x: pd.Series(x[0]['generated_text']).str.extract(r'SDG-(\d+)')[0]
# )

results_processed.head(3)

,cnt,text_id,sdg_true,result,label
0,0,6668,0,"""SDG-0""",0
1,1,6099,0,"""SDG-0""",0
2,2,6471,0,"""SDG-16""",16


In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,sdg_true,result,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,0,"""SDG-0""",0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,0,"""SDG-0""",0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,0,"""SDG-16""",16


In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

[]

In [ ]:
final  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.24      0.28      0.26        60
           1       0.58      0.63      0.61        60
           2       0.88      0.62      0.73        60
           3       0.75      0.68      0.71        60
           4       0.78      0.87      0.82        60
           5       0.87      0.65      0.74        60
           6       0.91      0.80      0.85        60
           7       0.84      0.62      0.71        60
           8       0.48      0.45      0.47        60
           9       0.63      0.68      0.66        60
          10       0.68      0.53      0.60        60
          11       0.60      0.57      0.58        60
          12       0.47      0.85      0.60        60
          13       0.70      0.73      0.72        60
          14       0.98      0.72      0.83        60
          15       0.71      0.82      0.76        60
          16       0.74      0.75      0.74        60
          17     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test

In [ ]:
results_processed_merged = final

In [ ]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['label']) or row['label'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'label'] = 0  # new_value

In [ ]:
results_processed_merged.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,sdg_true,result,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,0,"""SDG-0""",0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,0,"""SDG-0""",0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,0,"""SDG-16""",16


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.22      0.28      0.25        60
           1       0.58      0.63      0.61        60
           2       0.88      0.62      0.73        60
           3       0.75      0.68      0.71        60
           4       0.78      0.87      0.82        60
           5       0.87      0.65      0.74        60
           6       0.91      0.80      0.85        60
           7       0.84      0.62      0.71        60
           8       0.48      0.45      0.47        60
           9       0.63      0.68      0.66        60
          10       0.68      0.53      0.60        60
          11       0.60      0.57      0.58        60
          12       0.47      0.85      0.60        60
          13       0.70      0.73      0.72        60
          14       0.98      0.72      0.83        60
          15       0.71      0.82      0.76        60
          16       0.74      0.75      0.74        60

    accuracy    

In [ ]:
output_datapath = output_data_path

In [ ]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [ ]:
report_df

,precision,recall,f1-score,support
0,21.794872,28.333333,24.637681,60.000000
1,58.461538,63.333333,60.800000,60.000000
2,88.095238,61.666667,72.549020,60.000000
3,74.545455,68.333333,71.304348,60.000000
4,77.611940,86.666667,81.889764,60.000000
5,86.666667,65.000000,74.285714,60.000000
6,90.566038,80.000000,84.955752,60.000000
7,84.090909,61.666667,71.153846,60.000000
8,48.214286,45.000000,46.551724,60.000000
9,63.076923,68.333333,65.600000,60.000000


In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

69.4118007424337 66.17647058823529 66.17647058823529 66.85892950924386


In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

69.4 66.2 66.2 66.9
